In [1]:
import pyodbc
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix


from texttransformation import StringTransform, RowTextTransform, TransformDataset
from datasetbuilder import DatasetBuilder
from metrics import MetricsCalculator
from helpers import SaveModel, RestoreModel, Normalize, RemoveOutliers
from trainer import Trainer

Configure parameters 

In [2]:
DATASOURCE_COLUMNS = ['FindCode', 'Name', 'Phone', 'Fax', 'OtherPhone', \
                                  'Email', 'WebsiteURL', 'MailingAddressFreeform', \
                                  'MailingAddressCity', 'MailingAddressPostalCode', 'MailingAddressState']
DATASOURCE_INDEX = 'FindCode'
ONE_HOT_ENCONDING_COLUMNS = {'MailingAddressState': ['nunavut ',
                                                        'saskatchewan',
                                                        'ontario  ',
                                                        'alberta',
                                                        'british columbia ',
                                                        'prince edward island ',
                                                        'yukon territory',
                                                        'newfoundland',
                                                        'northwest territories',
                                                        'new brunswick',
                                                        'manitoba',
                                                        'nova scotia ',
                                                        'quebec']}
TEXT_METRICS = ['ratio','partial_ratio','token_sort_ratio','token_set_ratio','distance',
                'l_ratio','jaro','jaro_winkler','setratio','seqratio','longestnumericseq']
PASS_THROUGH_COMULNS = ['FindCode','MailingAddressState']
HIGH_IMPORTANCE_COLUMNS = ['Name']

alteration_rules = [
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['alpha', 2, 'replace'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [2],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    },
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['any', 2, 'add'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [4],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    },
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['digits', 1, 'add'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [3],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    }
]

COLUMN_ALTERATIION_RULES = []
for i in range(0, len(alteration_rules)):
    COLUMN_ALTERATIION_RULES.append(
        {
            1: alteration_rules[i],
            2: alteration_rules[i],                    
            3: alteration_rules[i],
            4: alteration_rules[i],
            5: alteration_rules[i],
            6: alteration_rules[i],
            7: alteration_rules[i],
            8: alteration_rules[i],
            9: alteration_rules[i]
        })

Fetch data from source. Meta should match configuration above

In [3]:
df_source = pd.read_csv('train_input.csv', header=0, names=DATASOURCE_COLUMNS)
df_source.FindCode = df_source.FindCode.astype(str).str.lower()
df_source.Name = df_source.Name.astype(str).str.lower()
df_source.Phone = df_source.Phone.astype(str).str.lower()
df_source.Fax = df_source.Fax.astype(str).str.lower()
df_source.OtherPhone = df_source.OtherPhone.astype(str).str.lower()
df_source.Email = df_source.Email.astype(str).str.lower()
df_source.WebsiteURL = df_source.WebsiteURL.astype(str).str.lower()
df_source.MailingAddressFreeform = df_source.MailingAddressFreeform.astype(str).str.lower()
df_source.MailingAddressCity = df_source.MailingAddressCity.astype(str).str.lower()
df_source.MailingAddressPostalCode = df_source.MailingAddressPostalCode.astype(str).str.lower()
df_source.MailingAddressState = df_source.MailingAddressState.astype(str).str.lower()

df_source = df_source[0:10]

Create training dataset

In [4]:
builder = DatasetBuilder(DATASOURCE_COLUMNS, DATASOURCE_INDEX, 
                         ONE_HOT_ENCONDING_COLUMNS, TEXT_METRICS, 
                         PASS_THROUGH_COMULNS, COLUMN_ALTERATIION_RULES,
                         HIGH_IMPORTANCE_COLUMNS, 8)
training_df = builder.generateTrainingDataset(df_source)

2020-04-17 19:52:40,968 - root - INFO - Column: Name_x, ratio metric took: 1.199622631072998 seconds
2020-04-17 19:52:42,359 - root - INFO - Column: Name_x, partial_ratio metric took: 1.3897902965545654 seconds
2020-04-17 19:52:43,533 - root - INFO - Column: Name_x, token_sort_ratio metric took: 1.1734352111816406 seconds
2020-04-17 19:52:44,692 - root - INFO - Column: Name_x, token_set_ratio metric took: 1.15958833694458 seconds
2020-04-17 19:52:45,830 - root - INFO - Column: Name_x, distance metric took: 1.1378860473632812 seconds
2020-04-17 19:52:46,990 - root - INFO - Column: Name_x, l_ratio metric took: 1.1593494415283203 seconds
2020-04-17 19:52:48,187 - root - INFO - Column: Name_x, jaro metric took: 1.1972429752349854 seconds
2020-04-17 19:52:49,327 - root - INFO - Column: Name_x, jaro_winkler metric took: 1.139528512954712 seconds
2020-04-17 19:52:50,456 - root - INFO - Column: Name_x, setratio metric took: 1.1287596225738525 seconds
2020-04-17 19:52:51,598 - root - INFO - Col

2020-04-17 19:54:23,411 - root - INFO - Column: MailingAddressFreeform_x, setratio metric took: 1.7191059589385986 seconds
2020-04-17 19:54:24,835 - root - INFO - Column: MailingAddressFreeform_x, seqratio metric took: 1.422792673110962 seconds
2020-04-17 19:54:26,625 - root - INFO - Column: MailingAddressFreeform_x, longestnumericseq metric took: 1.7891409397125244 seconds
2020-04-17 19:54:28,689 - root - INFO - Column: MailingAddressCity_x, ratio metric took: 2.0546836853027344 seconds
2020-04-17 19:54:30,457 - root - INFO - Column: MailingAddressCity_x, partial_ratio metric took: 1.7663295269012451 seconds
2020-04-17 19:54:31,963 - root - INFO - Column: MailingAddressCity_x, token_sort_ratio metric took: 1.5044033527374268 seconds
2020-04-17 19:54:33,417 - root - INFO - Column: MailingAddressCity_x, token_set_ratio metric took: 1.4518253803253174 seconds
2020-04-17 19:54:35,350 - root - INFO - Column: MailingAddressCity_x, distance metric took: 1.9329149723052979 seconds
2020-04-17 

In [5]:
trainer = Trainer(training_df, 0.2)
SaveModel(trainer.execute(), "TrainedModel.sav")

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
Algorithm                                    LogisticRegression
Time                                                    2.45878
Parameters          {'clf__C': 0.4, 'clf__fit_intercept': True}
Estimator     (LogisticRegression(C=0.4, cla